# FireCrawl playpen

This is a simple notebook to discover what the response of `Firecrawl`'s response object looks like...

The documentation takes time... and I got a bit unpatient... :)

In [1]:
from firecrawl import Firecrawl
import dotenv, os, re, ast
import urllib.parse
import hashlib


dotenv.load_dotenv(dotenv.find_dotenv("firecrawl-flink_docs/.env"))
firecrawl = Firecrawl(api_key=os.getenv('FIRECRAWL_API_KEY'))

## /crawl

In [22]:
print("\n Starting crawl...")

# Crawl with scrape options
response = firecrawl.crawl('https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/',
    limit=3,
    scrape_options={
        "maxDepth": 1,
        "render": False,
        "ignoreRobotsTxt": True,
    }
)



print("\n Crawl finished...")

print("\n Crawl response:")
print(response.model_dump())


 Starting crawl...

 Crawl finished...

 Crawl response:
{'status': 'completed', 'total': 0, 'completed': 0, 'credits_used': 0, 'expires_at': datetime.datetime(2026, 1, 4, 11, 24, 44, tzinfo=TzInfo(0)), 'next': None, 'data': []}


The above shows that the crawl does not really work. I suspect it has to do with the `robots.txt` restriction on flinks docs... Not sure why that is restricted...

## /scrape

In [5]:
print("\n Starting scrape...")

# Crawl with scrape options
response = firecrawl.scrape(
    url='https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/',
    wait_for=2000,
    only_main_content=True,
    formats=['markdown'],
)



print("\n Scrape finished...")

print('\n Writing to file...')
with open("./flink_firecrawl_output.json", "w", encoding="utf-8") as f:
    f.write(response.model_dump()['markdown'])

print("\n Scrape response:")
print(response.model_dump()['markdown'][:100])




 Starting scrape...

 Scrape finished...

 Writing to file...

 Scrape response:
# Concepts  [\#](https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/\


This prints the markdown content of the scraped page. I.e. it works!!! YES!!!

# Metadata extraction

In [2]:
with open('./data/flink_firecrawl_markdown.md', 'r') as f:
    lines = f.readlines()

md_content = '\n'.join(lines)

with open('./data/flink_firecrawl_response_full.txt', 'r', encoding='utf-8') as f:
    full_content = f.read()

file_response = ast.literal_eval(full_content)

## Datamodel

In this part we are describing the data that needs to be saved from the scraping per page.

1. Main content into `.md`-file:
    1. File name = `<prefix>_<page_id>.md`
        1. `<prefix>` = url - `<https://../docs/>`
        2. `<page_id>` = hash of `<prefix>`
2. Meta-data:
    1. page_id: hash
    2. title: str
    3. url: str
    4. parent_url: str
    5. is_root_url: bool
    6. child_urls (a list of tuples for ('link_text','link_url')): list[(str,str)]
    7. scrape_timestamp: timestamp



Here are some suggestions from Copilot. It requires some work from an LLM (especially in the summary and stubb parts etc - but lets check if we can integrate this into ollama - i.e. not going out to external LLMs).

The name of the file is interesting:
> Suggested filename (example): overview_019b8f59-6e02.md
> (If you prefer canonical-hash, replace the UUID prefix with sha256(canonical_url)[:12].)
> Main .md file contents (save exactly as file body; no frontmatter):
>

Here is the json output:
```
{
"page_id": "sha256:<hex-of-canonical-url>",
"content_hash": "sha256:<hex-of-normalized-markdown>",
"slug": "overview",
"title": "Overview | Apache Flink",
"url": "https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/",
"canonical_url": "https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/",
"scrape_id": "019b8f59-6e02-767d-bf46-0690425307de",
"index_id": "2a63f795-1f18-4d10-a6c2-474de4abeab9",
"status_code": 200,
"content_type": "text/html",
"language": "en",
"summary": "Overview of Flink concepts, APIs, and training resources.",
"headings": [{"level":1,"text":"Concepts"},{"level":2,"text":"Flink’s APIs"}],
"assets": [{"original_url":"https://nightlies.apache.org/flink/.../fig/levels_of_abstraction.svg","inferred_filename":"levels_of_abstraction.svg","content_type":"image/svg+xml"}],
"previous_url": null,
"next_urls": [],
"is_root_url": false,
"scrape_timestamp": "2026-01-05T12:00:00Z",
"cached_at": null,
"provenance": "nightlies.apache.org",
"notes": "content taken from Firecrawl response.model_dump(); consider canonical_url normalization before dedup."
}
```

In [16]:
from models.metadata import PageMetadata

class Tester:

    def __init__(self,root_url: str = None):
        if root_url:
            self.root_url = root_url


    def extract_prefix(self, url, remove_start: str = 'https://nightlies.apache.org/', remove_end: str = '/docs/') -> str:
        pattern = re.compile(re.escape(remove_start) + r'.*?' + re.escape(remove_end))
        rest = pattern.sub('', url, 1)
        cleaned = re.sub(r'[^A-Za-z]+', '_', rest).strip('_')
        return re.sub(r'_+', '_', cleaned)
    
    def prefix_to_hash(self, prefix: str, numeric: bool = False):
        h = hashlib.sha256(prefix.encode('utf-8')).hexdigest()
        return int(h[:16], 16) if numeric else h
    
    def _normalize_url(self, url: str) -> str:
        parts = urllib.parse.urlsplit(url)

        # normalize scheme and netloc, remove fragment
        scheme = parts.scheme.lower()
        netloc = parts.netloc.lower()
        if (scheme == 'http' and netloc.endswith(':80')) or (scheme == 'https' and netloc.endswith(':443')):
            netloc = netloc.rsplit(':', 1)[0]

        normalized = urllib.parse.urlunsplit((scheme, netloc, parts.path or '/','',''))
        normalized = normalized.rstrip('/')

        return normalized
    
    
    def extract_markdown_links(self, text):
        """
        Extract unique markdown page links (text, url) from `text`, excluding image links.
        Fragments (anchors) are removed so multiple section links to the same page yield one entry.
        """

        pattern = re.compile(r'(?<!\!)\[(?P<text>[^\]]+)\]\((?P<url>https?://[^\s)]+)\)')
        seen = set()
        ret = []

        for m in pattern.finditer(text):
            raw_url = m.group('url').replace('\\', '')
            parts = urllib.parse.urlsplit(raw_url)

            # normalize scheme and netloc, remove fragment
            normalized = self._normalize_url(raw_url)

            if (normalized in seen) or (normalized == self._normalize_url(self.root_url)):
                continue
            seen.add(normalized)

            desc = re.sub(r'\s+', ' ', m.group('text')).strip()
            desc = re.sub(r'[^A-Za-z\s]+', '', desc)
            desc = re.sub(r'\s+', ' ', desc).strip()
            ret.append((desc, normalized))

        return ret
    
    def save_markdown_file(self, data: dict, content: str, save_dir: str = './data/markdown_files/'):
        file_name = f"{save_dir}{data.get('prefix')}_{data.get('page_id')}.md"
        with open(file_name, "w", encoding="utf-8") as f:
            f.write(content)

        
    def parse_raw_response(self, raw_response: str,parent_url: str = None) -> dict:
        data_dict = {}

        data_dict['title'] = raw_response['metadata']['title']
        data_dict['url'] = raw_response['metadata']['url']
        if parent_url:
            data_dict['is_root_url'] = False
        else:
            data_dict['is_root_url'] = True
            self.root_url = data_dict['url']
        data_dict['parent_url'] = parent_url
        
        data_dict['prefix'] = self.extract_prefix(data_dict['url'])
        data_dict['page_id'] = self.prefix_to_hash(data_dict['prefix'])
        data_dict['child_urls'] = self.extract_markdown_links(raw_response['markdown'])

        return data_dict

    def process_response(self, raw_response: dict) -> dict:
        data_dict = self.parse_raw_response(raw_response)
        metadata = PageMetadata.model_validate(data_dict)
        self.save_markdown_file(data_dict, raw_response['markdown'])
        
        return metadata
    
    

ts = Tester()

In [17]:
# ts.extract_prefix(response.model_dump()['metadata']['url'])
ts.extract_prefix(file_response['metadata']['url'])
# ts.extract_prefix('https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/operators/')
# ts.extract_prefix('https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/operators/#keyed-and-non-keyed-operators')

'concepts_overview'

In [18]:
ts.process_response(file_response)

< PageMetadata page_id=d699b5373c84d3776703d9c89d472a1ecee196e604219eb74f8e5647e6a4513c,
  url=https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview
  title=Overview | Apache Flink,
  is_root_url=True,
  parent_url=None,
  child_urls[7]=
  -->  Handson Training (https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/overview)
  -->  Data Pipelines ETL (https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/etl)
  -->  Fault Tolerance (https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/fault_tolerance)
  -->  Streaming Analytics (https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/streaming_analytics)
  -->  DataStream API (https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/overview)
  -->  Process Function (https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/operators/process_function)
  -->  Table API (https://nig

In [19]:
ts.parse_raw_response(file_response)

{'title': 'Overview | Apache Flink',
 'url': 'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/',
 'is_root_url': True,
 'parent_url': None,
 'prefix': 'concepts_overview',
 'page_id': 'd699b5373c84d3776703d9c89d472a1ecee196e604219eb74f8e5647e6a4513c',
 'child_urls': [('Handson Training',
   'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/overview'),
  ('Data Pipelines ETL',
   'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/etl'),
  ('Fault Tolerance',
   'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/fault_tolerance'),
  ('Streaming Analytics',
   'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/learn-flink/streaming_analytics'),
  ('DataStream API',
   'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/overview'),
  ('Process Function',
   'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/o

In [5]:
%ll

total 36
drwxrwxr-x 2 joestry  4096 Jan  6 20:24 data/
drwxrwxr-x 2 joestry  4096 Jan  7 20:50 models/
-rw-rw-r-- 1 joestry 18213 Jan  8 20:50 play-pen.ipynb
-rw-rw-r-- 1 joestry   838 Jan  2 15:46 run-crawling.py
-rw-rw-r-- 1 joestry   624 Dec 28 09:57 run-scraping.py
